In [1]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import lightgbm as lgb
from derive_functions.derive_date_var import derive_calender_feats
from derive_functions.data_prepare_func import create_dt,reduce_mem_usage
from derive_functions.derive_lag_mean_feats import create_lag_feats,create_lag_mean_feats
from derive_functions.derive_mean_feats_cat import get_df_cat
from derive_functions.derive_mean_feats_dept import get_df_dept
from derive_functions.derive_mean_feats_id import get_df_id
from derive_functions.derive_mean_feats_item import get_df_item
from derive_functions.derive_mean_feats_state import get_df_state
from derive_functions.derive_mean_feats_store import get_df_store
from derive_functions.derive_deviation_feats import get_deviation_feats
import gc

# 1. load data

In [2]:
# 数据加载
data = pd.read_pickle('data/processed_data/df_test_var_all_0626.pkl')
# data_train = data.copy(deep=True).reset_index(drop=True)
# data_train = data[data.date<='2016-04-24'].copy(deep=True).reset_index(drop=True)
data_test = data[(data.date>='2015-12-22') & (data.state_id=='CA')].copy(deep=True).reset_index(drop=True)
del data
gc.collect()

0

In [3]:
recursive_feats=[]
for i in data_test.columns:
    if ('lag_7' in i) or ('shift7' in i):
        recursive_feats.append(i)
data_test = data_test.sort_values(by=['id','date'],ascending=[True,True]).reset_index(drop=True)

In [4]:
useless_feats = ['d','date','wm_yr_wk','sales','weekday','revenue','id','state_id']
cat_feats = ['item_id', 'dept_id','cat_id','store_id'] + ["wday","event_name_1", "event_name_2", "event_type_1", "event_type_2"]
train_cols = data_test.columns[~data_test.columns.isin(useless_feats)]

# 2. load model and predict sales for the test period

In [ ]:
# 模型加载
m_lgb_v8 = lgb.Booster(model_file='model/time_split/model_tweedie_CA_var_all_v8.pkl')


fday = datetime(2016,5,23) 
alphas = [1,1.023, 1.028,1.018]
# v1,v2,v3,v4
for i in range(len(alphas)):
    print(alphas[i])
    df_test = data_test.copy(deep=True)
    for tdelta in range(0,28):
        day = fday + timedelta(days=tdelta)
        print(day)
        print(datetime.now())
        df_test.loc[df_test.date==day,'sales'] = alphas[i] * m_lgb_v8.predict(df_test.loc[df_test.date == day, train_cols])
        df_test['revenue'] = df_test['sales'] * df_test['sell_price']
        df_test = df_test.sort_values(by=['id','date'],ascending=[True,True]).reset_index(drop=True)
        df_test = create_lag_feats(df_test, recursive=True)
        df_test = create_lag_mean_feats(df_test, recursive=True)
    cols = [f"F{i}" for i in range(1, 29)]
    te_sub = df_test.loc[df_test.date >= fday, ["id", "sales"]].copy()
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")[
        "id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F"]).unstack()[
        "sales"][cols].reset_index()
    te_sub.fillna(0., inplace=True)
    te_sub.sort_values("id", inplace=True)
    te_sub.reset_index(drop=True, inplace=True)
    sub2 = te_sub.copy()
    sub2["id"] = sub2["id"].str.replace("evaluation$", "validation")
    sub_all = pd.concat([te_sub, sub2], axis=0, sort=False)  
    sub_all.to_csv("result/test/CA_v8_submission_v"+str(i+1)+".csv",index=False)